# Pybfe getting started

This notebook illustrates Pybfe usage. 

It focuses on pybfe. Almost all pybatfish (open source) APIs will work with pybfe in their sessionized versions. For example:
 * `bf_init_snapshot()` => `bf.init_snapshot()`   (`bf` is the Session object)
 * `bfq.nodeProperties()` => `bf.q.nodeProperties()` 

**NB: This notebook uses many experimental features that are subject to change.**

## Creating a session with BFE service

In [2]:
import os
from pybfe.client.session import Session

# setup cert if needed
os.environ['BFE_SSL_CERT'] = "nexbfe.crt"

BFE_HOST = "batfish.nexariacloud.com"
BFE_PORT = 443

NETWORK_NAME="bfe-demo"
SNAPSHOT_NAME="baseline"
SNAPSHOT_DIR="snapshot" 

bf = Session(host=BFE_HOST, port=BFE_PORT)

If you get "Exception: Can only establish a session with a Batfish Enterprise backend" that could mean that your SSL cert is not properly setup. If you can connect to https://BFE_HOST:BFE_PORT/ using the browser, it is likely a cert setup issue.

## Initializing the network

In [3]:
# Define the network that we'll be working with

## first delete the network if it already exists
networks = bf.list_networks()
if NETWORK_NAME in networks:
    bf.delete_network(NETWORK_NAME)

bf.set_network(NETWORK_NAME)

'bfe-demo'

### Defining network aggregates 

This is an optional step that helps produce good topology layout and more understandable answers for NMAP and Reachability comparison.

In [4]:
# the definitions below are over the snapshot bundled with this notebook
AGGREGATES = {
    "aggregates": [
        {
            # name of the aggregate
            "name": "SJC",
            # devices in the aggregate, specified using globs over device names
            "patterns": ["bor*", "fwl*", "spine*", "leaf*", "bl*"],
            # child aggregates, recursively defined in a similar manner
            "children": [
                {
                    "name": "border", 
                    "patterns": ["bor*"]
                },
                {
                    "name": "spine", 
                    "patterns": ["spine*"]
                },
                {
                    "name": "leafs", 
                    "patterns": ["leaf*"]},
                {
                    "name": "border leafs", 
                    "patterns": ["bl*"]
                },
            ],
        }
    ]
}

# put the aggregate definitions on the server.
bf.put_network_aggregates(AGGREGATES)

##### If you change aggregate definitions after initializing snapshots in the network, new definitions will be used only for the snapshots initialized after the change. The layout of old snapshots will not reflect new aggregate definitions.

## Defining network policies

Network policies can be specified using JSON/YAML. You can view the list of supported policies on the Dashboard. 

You can get the JSON for any policy by setting it up on the Dashboard and then viewing its parameters. The "View parameters" feature is in the dropdown with "View results" when you are viewing policy output.

In [5]:
# lets define an example policy
bgp_session_establishment = {
  "description": "BGP peers must be able to establish TCP connections",
  "title": "BGP session establishment validation",
  "bgp_session_establishment": {}
}

bf._experimental_create_policy(bgp_session_establishment)

uid: "b01bbcd1-4ff2-44c8-a1cf-5f221dd0dfaa"

You can define any number of network policies by repeating the steps above. 

**Notes:**
 1. You may define additional policies after some snapshots have been initialized in the network. However, those new policies will be automatically evaluated only for future snapshots. 
 2. Deleting network policies is not possible at the moment. If you want to clean up some policies, delete and create a new network. 

## Initializing the snapshot

In [6]:
bf.init_snapshot(SNAPSHOT_DIR, name=SNAPSHOT_NAME, overwrite=True)

'baseline'

## Fetching policy results via API

You can view the results of policy evaluation on the Dashboard. You can also fetch them programmatically as below.

In [7]:
from intentionet.bfe.proto import api_gateway_pb2 as api
from intentionet.bfe.proto import policies_api_pb2 as policies_api

def policy_status_to_string(status):
    if status == policies_api.POLICY_STATUS_UNKNOWN:
        return "UNKNOWN"
    elif status == policies_api.POLICY_STATUS_FAIL:
        return "FAIL"
    elif status == policies_api.POLICY_STATUS_PASS:
        return "PASS"
    elif status == policies_api.POLICY_STATUS_PASS_WARN:
        return "WARN"
    else:
        raise RuntimeError("Unknown policy status {}".format(status))


def get_policy_results(bf: Session):
    """
    Get policy evaluation results for a snapshot.
    
    Returns a map from policy title to status
    """
    response = bf._api_gw.ListPolicyResultsMetadata(
        api.ListPolicyResultsMetadataRequest(
            network_name=bf.network, snapshot_name=bf.snapshot
        )
    )
    status = {}
    for result in response.metadata:
        policy_response = bf._api_gw.GetPolicy(
            api.GetPolicyRequest(
                network_name=bf.network, policy_id=result.policy_id
            )
        )
        status[policy_response.policy.input.title] = policy_status_to_string(result.status)

    return status

get_policy_results(bf)

{'BGP session establishment validation': 'PASS'}

### Getting Compare results via API

In [8]:
# initialize another snapshot for comparison
NEW_SNAPSHOT_NAME="new_snapshot"
NEW_SNAPSHOT_DIR="new_snapshot" 
bf.init_snapshot(NEW_SNAPSHOT_DIR, name=NEW_SNAPSHOT_NAME, overwrite=True)

'new_snapshot'

In [10]:
def get_compare_results(bf: Session, snapshot_name: str, reference_snapshot_name: str):
    """
    Gets snapshot comparison results.
    
    Returns a map from comparison type to either a count of changes or whether that type has changed.
    """
    resp = bf._api_gw.GetSnapshotComparisonMetadata(
        api.GetSnapshotComparisonMetadataRequest(
            network_name=bf.network,
            snapshot_name=snapshot_name,
            reference_snapshot_name=reference_snapshot_name
        )
    )

    result = {}
    
    #prints all available categories
    #print(resp) 
 
    # pick some categories to embed in the result 
    result["devices_configuration"] = resp.configurations.num_results
    result["interfaces"] = resp.interfaces.num_results
    result["devices_routing"] = resp.routes.num_results
    #result["increased_flows"] = resp.reachability.has_increased_flows
    #result["decreased_flows"] = resp.reachability.has_decreased_flows

    return result

get_compare_results(bf, NEW_SNAPSHOT_NAME, SNAPSHOT_NAME)

{'devices_configuration': 61, 'interfaces': 671, 'devices_routing': 37}

In [11]:
def get_compare_config_summary_results(bf: Session, snapshot_name: str, reference_snapshot_name: str):
    
    """
    Gets snapshot comparison results.
    
    Returns a map from comparison type to either a count of changes or whether that type has changed.
    """
    resp = bf._api_gw.GetSnapshotComparisonRoutingProtocolsOspfProcess(
        api.GetSnapshotComparisonRoutingProtocolsOspfProcessRequest(
            network_name=bf.network,
            snapshot_name=snapshot_name,
            reference_snapshot_name=reference_snapshot_name
        )
    )
    
    return resp

In [12]:
get_compare_config_summary_results(bf, NEW_SNAPSHOT_NAME, SNAPSHOT_NAME)

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNIMPLEMENTED
	details = "Method not implemented!"
	debug_error_string = "{"created":"@1622311822.195000000","description":"Error received from peer ipv4:54.197.65.81:443","file":"src/core/lib/surface/call.cc","file_line":1068,"grpc_message":"Method not implemented!","grpc_status":12}"
>

## Running tests (without creating policies)

In some scenarios, it is helpful to run tests on the snapshots without creating a policy. The same JSON definiitions that is used for policies, can also be run against network snapshots

In [9]:
# get the full output of the test
bf.asserts.run_assertion(bgp_session_establishment)

id {
  uid: "0ee4c47d-752d-4c93-b355-51cb33ca6266"
}
result {
  metadata {
    status: ASSERTION_STATUS_PASS
    stats {
      pass: 12
    }
  }
  bgp_session_establishment {
    violator_descriptions: "Cannot establish a TCP connection."
  }
}

In [10]:
# get only the pass/fail status
bf.asserts.assert_that(bgp_session_establishment)

True